### 1. Import packages

In [1]:
import wandb
wandb.login()

wandb: Currently logged in as: rant3. Use `wandb login --relogin` to force relogin


True

In [2]:
import numpy as np
import pandas as pd
import random
from IPython import display
from collections import namedtuple, deque
import time
from tqdm import tqdm
import matplotlib.pyplot as plt
%matplotlib inline
""
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
device = torch.device("cpu")
# use cpu run
import gym

### 2. Helper functions

In [3]:
def dict2array(state):
    new_state = []
    for key  in state.keys():
        if key != 'sw':
            new_state.append(state[key])
        else:
            new_state += list(state['sw'])        
    state = np.asarray(new_state)
    return state

### 3. Initialize the environment

In [5]:
env_args = {
    'run_dssat_location': '/opt/dssat_pdi/run_dssat',  # assuming (modified) DSSAT has been installed in /opt/dssat_pdi
    'log_saving_path': './logs/dssat-pdi.log',  # if you want to save DSSAT outputs for inspection
    # 'mode': 'irrigation',  # you can choose one of those 3 modes
    # 'mode': 'fertilization',
    'mode': 'all',
    'seed': 123456,
    'random_weather': False,  # if you want stochastic weather
}
env = gym.make('gym_dssat_pdi:GymDssatPdi-v0', **env_args)
print('Observation:',env.observation,)
print(len(env.observation),len(env.observation['sw']))
ram_dimensions = 35
#ram_dimnetion = 9 if partial
nb_actions = 5
print('\nRam information received from DASSAT will has %d dimensions.' % ram_dimensions)
print('There are %d possible actions at each step.' % nb_actions)
print('Discrete?',type(gym.spaces)== gym.spaces.Discrete)
# observation has 27 elements, 9 values in soil water
# state size = 27+8 dimension
# how to defind nb_action? why is 200?

Observation: {'cleach': 0.0, 'cnox': 0.0, 'cumsumfert': 0.0, 'dap': 0, 'dtt': 0.0, 'es': 0.0, 'grnwt': 0.0, 'istage': 7, 'nstres': 0.0, 'pcngrn': 0.0, 'pltpop': 7.199999809265137, 'rain': 0.0, 'rtdep': 0.0, 'runoff': 0.0, 'srad': 13.300000190734863, 'sw': array([0.086     , 0.086     , 0.086     , 0.086     , 0.086     ,
       0.076     , 0.076     , 0.13      , 0.25799999]), 'swfac': 0.0, 'tleachd': 0.0, 'tmax': 22.200000762939453, 'tmin': 3.299999952316284, 'tnoxd': 0.0, 'topwt': 0.0, 'trnu': 0.0, 'vstage': 0.0, 'wtdep': 0.0, 'wtnup': 0.0, 'xlai': 0.0}
27 9

Ram information received from DASSAT will has 35 dimensions.
There are 5 possible actions at each step.
Discrete? False


### 4. Define the network

In [6]:
class QNetwork(nn.Module):
    """Agent (Policy) Model."""
    # given a state of 35 dim, Qnetwork will return 200 values for each possible action  

    def __init__(self, state_size, action_size, fc1_units=256):
        """Initialize parameters and build model.
        Params
        ======
            state_size (int): Dimension of each state
            action_size (int): Dimension of each action
            fc1_units (int): Number of nodes in first hidden layer
            why is it 256? randomly?
        """
        super(QNetwork, self).__init__()
        self.fc1 = nn.Linear(state_size, fc1_units)
        self.fc2 = nn.Linear(fc1_units, action_size)
        # set a nn with 1 layer
        
    def forward(self, state):
        """Build a network that maps state -> action values."""
        x = F.relu(self.fc1(state))
        #Applies the rectified linear unit function element-wise. max(0,x)
        return self.fc2(x)

### 5. Define the Replay Buffer

In [7]:
class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            how many samples stored in buffer
            batch_size (int): size of each training batch
        """
        self.action_size = action_size
        self.memory = deque(maxlen=buffer_size)
        # store [s,a,r,s',done] for each sample in buffer
        self.batch_size = batch_size
        self.experience = namedtuple("Experience", field_names=["state", "action", "reward", "next_state", "done"])
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
#         action = dict2array(action)
#         print('transfored actions:',action)
        e = self.experience(state, action, reward, next_state, done)
        self.memory.append(e)
    
    def sample(self):
        """Randomly sample a batch of experiences from memory."""
        experiences = random.sample(self.memory, k=self.batch_size)
#         for e in experiences:
# #             print(e)
# #             print(e.state.shape, e.next_state.shape)
#             break
        states = torch.from_numpy(np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        # vstack is add rows together as a single matrix
        actions = torch.from_numpy(np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        rewards = torch.from_numpy(np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        next_states = torch.from_numpy(np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        dones = torch.from_numpy(np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)
        #get all the states, actions, rewards, next_states, dones for all elements in this sample batch, each as a single matrix
        #device here is cpu?
        return (states, actions, rewards, next_states, dones)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

### 6. Define the Agent

In [8]:
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 60                # for update of target network parameters
LR = 1e-5               # learning rate 
UPDATE_EVERY = 4        # how often to update the network

In [9]:
class Agent():
    """Interacts with and learns from the environment."""

    def __init__(self, state_size, action_size):
        """Initialize an Agent object.
        Params
        ======
            state_size (int): dimension of each state
            action_size (int): dimension of each action
        """
        self.state_size = state_size
        self.action_size = action_size

        # Q-Network
        self.qnetwork_local = QNetwork(state_size, action_size).to(device)
        self.qnetwork_target = QNetwork(state_size, action_size).to(device)
        self.optimizer = optim.Adam(self.qnetwork_local.parameters(), lr=LR)

        # Replay memory
        self.memory = ReplayBuffer(action_size, BUFFER_SIZE, BATCH_SIZE)
        # Initialize time step (for updating every UPDATE_EVERY steps)
        self.t_step = 0
    
    def step(self, state, action, reward, next_state, done):
        # Save experience in replay memory
        # add current (s,r,a,s') and sample a batch and learn if update_every
        self.memory.add(state, action, reward, next_state, done)
        
        # Learn every UPDATE_EVERY time steps.
        self.t_step += 1
        if self.t_step%UPDATE_EVERY == 0:
            # If enough samples are available in memory, get random subset and learn
            if len(self.memory) > BATCH_SIZE:
                experiences = self.memory.sample()
                self.learn(experiences, GAMMA)

    def act(self, state, eps=0.):
        """Returns actions for given state as per current policy.
        Params
        ======
            state (array_like): current state
            eps (float): epsilon, for epsilon-greedy action selection
        """
        state = torch.from_numpy(state).float().unsqueeze(0).to(device)
        self.qnetwork_local.eval()
        with torch.no_grad():
            action_values = self.qnetwork_local(state)
        self.qnetwork_local.train()
#         Epsilon-greedy action selection
        if random.random() > eps:
            return np.argmax(action_values.cpu().data.numpy())
        # return the action index with the largest value
        else:
            return random.choice(np.arange(self.action_size))
            #return one action randomly with possibility eps

#         Epsilon-greedy action selection
#         if random.random() > eps:
#             return np.argmax(action_values.cpu().data.numpy())
#         else:
#             return random.choice(np.arange(self.action_size))
# #         return action_values.cpu().data.numpy()


    def learn(self, experiences, gamma):
        """Update value parameters using given batch of experience tuples.
        Params
        ======
            experiences (Tuple[torch.Variable]): tuple of (s, a, r, s', done) tuples 
            gamma (float): discount factor
        """
        states, actions, rewards, next_states, dones = experiences
        
        # Get max predicted Q values (for next states) from target model
        Q_targets_next = self.qnetwork_target(next_states).detach().max(1)[0].unsqueeze(1)
        # detach for diemnsion correctness
        # GET largest value of self.qnetwork_target(next_states)
#         print(Q_targets_next)
        # Compute Q targets for current states 
        Q_targets = rewards + (gamma * Q_targets_next * (1 - dones))
        # 0 if done

        # Get expected Q values from local model
#         print('out',self.qnetwork_local(states).shape)
        Q_expected = self.qnetwork_local(states).gather(1, actions)
        #Why need to gather? actions are indexs?

        # Compute loss
        loss = F.mse_loss(Q_expected, Q_targets)
        # Minimize the loss
        self.optimizer.zero_grad()
        loss.backward()
        # after this, the parameter of local network will change based on gradient descent
        for param in self.qnetwork_local.parameters():
            param.grad.data.clamp_(-1, 1)
        # stabilize traning to keep grad between (-1,1)
        self.optimizer.step()
        #固定用法, change learning rate

        #Update target network weights every TAU learning steps (so every TAU*UPDATE_EVERY t_step)
        if self.t_step%(TAU*UPDATE_EVERY)==0:
            self.update_target_net(self.qnetwork_local, self.qnetwork_target)                     

    def update_target_net(self, local_model, target_model):
        """Soft update model parameters.
        Params
        ======
            local_model (PyTorch model): weights will be copied from
            target_model (PyTorch model): weights will be copied to
        """
        target_model.load_state_dict(local_model.state_dict())
    def save(self):
        torch.save(self.qnetwork_local.state_dict(),'/home/rant3/focal/model.pth')

In [10]:
def get_reward_costant_k1(state, action, next_state, done, k1, k2, action_amount, i_episode):
    #if done, return Yield (topwt) - k*cost
    penalty = 0
    #k1=10**(-3)*(i_episode*2)
    # base cost is current input action
    if action_amount > 160:
        if action!=0:
            penalty = (action_amount-160)
    if done:
        reward = state[29] - k2*action - k1*penalty
        return reward
    #if done, return Yield (topwt) - k*cost
    else:
        reward = -k2*(action + state[25]) - k1*penalty
        return reward
    # otherwise, return -k*(action+leaching)

### DQN for nitrogen management

In [11]:
agent = Agent(state_size=ram_dimensions, action_size=nb_actions)

In [12]:
def dqn(n_episodes=2000, max_t=500, eps_start=1.0, eps_end=0, eps_decay=0.992, solved=1000, optimize = True,exp=1):
    """Deep Q-Learning.
    
    Params
    ======
        n_episodes (int): maximum number of training episodes
        max_t (int): maximum number of timesteps per episode
        eps_start (float): starting value of epsilon, for epsilon-greedy action selection
        eps_end (float): minimum value of epsilon
        eps_decay (float): multiplicative factor (per episode) for decreasing epsilon
    """
    start = time.time()                # Start time
    scores = []                        # list containing scores from each episode
    scores_window = deque(maxlen=100)  # last 100 scores
    list_eps = []
    eps = eps_start                    # initialize epsilon
    action_amount=0
    action_amount_list=[]
    action_amount_window = deque(maxlen=100)
    leaching_sum_list = []
    leaching_sum_window = deque(maxlen=100)
    y=0
    max_score=800
    wandb.init(
      # Set the project where this run will be logged
      project="Random weather", 
      # We pass a run name (otherwise it’ll be randomly assigned, like sunshine-lollypop-10)
      name=f"experiment_{exp}" )
      # Track hyperparameters and run metadata)
    for i_episode in range(1, n_episodes+1):
        # Reset env and score at the beginning of episode
        state = env.reset() # reset the environment and get current state
        state = dict2array(state)
        score = 0 # initialize the score
        action_amount = 0
        leaching_sum = state[25]
        for t in range(max_t):
            action1 = agent.act(state, eps) if optimize else 2
            action_amount = action_amount + action1*40
            action = {
                    'anfer': action1*40,  # if mode == fertilization or mode == all ; nitrogen to fertilize in kg/ha
                    'amir': 0,  # if mode == irrigation or mode == all ; water to irrigate in L/ha
            }
            next_state, reward, done, _ = env.step(action)      # send the action to the environment
            #reward = reward-
            # add changes to reward to penalize multiple actions
            # this step is different from agent step
            if done:
                y=state[29]*10
                next_state = state
                reward = get_reward_costant_k1(state, action['anfer'], next_state, done, 1, 0.1, action_amount, i_episode)
                agent.step(state, action1, reward, next_state, done)
                score += reward
                action_amount_list.append(action_amount)
                leaching_sum_list.append(leaching_sum)
                #print('Episode is ', i_episode, 'Yield is',state[29])
                action_amount_window.append(action_amount)
                leaching_sum_window.append(leaching_sum)
                break
            next_state = dict2array(next_state)
            reward = get_reward_costant_k1(state, action['anfer'], next_state, done, 1, 0.1, action_amount,i_episode)
            agent.step(state, action1, reward, next_state, done)
            state = next_state
            leaching_sum = leaching_sum + state[25]
            score += reward
            #append

        if score>max_score:
            if i_episode>800:
                max_score=score
                agent.save()
                print("max score is ",max_score)
        scores_window.append(score)       # save most recent score
        scores.append(score)              # save most recent score
        
        eps = max(eps_end, eps_decay*eps) # decrease epsilon
        list_eps.append(eps)
        #if i_episode > 1400:
        #    eps = 0
        if i_episode % 40 == 0:
            print('\rEpisode {}/{} \tAverage Score: {:.2f}'.format(i_episode, n_episodes, np.mean(scores_window)))
            print('Epsilon: {}'.format(eps))
            print('Average nitrogen amount is', np.mean(action_amount_window))
            print('Average leaching sum is', np.mean(leaching_sum_window))
            
        if np.mean(scores_window)>solved:
            print('Game Solved after {} episodes'.format(i_episode))
            break
        r_score=0.1*(y-action_amount-leaching_sum)
        wandb.log({"reward": score, "N_amount": action_amount,"leaching":leaching_sum,'yield':y,'real_socre':r_score})
    time_elapsed = time.time() - start
    print("Time Elapse: {:.2f} seconds".format(time_elapsed))
    
    return scores, list_eps, action_amount_list, leaching_sum_list

In [13]:
scores, list_eps, action_amount_list, leaching_sum_list = dqn(n_episodes=850,exp=1)

Episode 40/850 	Average Score: -568906.05
Epsilon: 0.7252151801981999
Average nitrogen amount is 10730.0
Average leaching sum is 2888.7063525510953
Episode 80/850 	Average Score: -428628.07
Epsilon: 0.5259370575899076
Average nitrogen amount is 9166.0
Average leaching sum is 2493.2794895414963


KeyboardInterrupt: 

In [ ]:
list_eps

In [12]:
import pandas as pd
#print(len(scores))
#action_list=action_list/100
a = pd.DataFrame(scores)
a.to_csv('4000 episode_large buffer.csv', index=False, header=False)
#b = pd.DataFrame(action_list)
#b.to_csv('Action with constant k2=0.5.csv', index=False, header=False)

### Given a constant input: reward = 19847

In [14]:
import matplotlib.pyplot as plt
plt.figure()
plt.plot(scores[-3000:])
plt.title("3000")
plt.xlabel("Traing Steps")
plt.ylabel("Cumulative Rewards")
plt.savefig("3000.png")

In [18]:
plt.plot(scores)
plt.yscale('symlog')
plt.title("Nitrogen Management")
plt.xlabel("Traing Steps")
plt.ylabel("Cumulative Rewards")
plt.savefig("Cumulative reward (log scale) linear penalty (partial observability).png")

In [18]:
plt.figure()
print(action_list)
plt.plot(action_list)
plt.title("Naction of last episode")
plt.xlabel("Days")
plt.ylabel("Nitrogen input")
plt.savefig("Action with constant k2=0.5.png")

[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.1  0.9  0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.04
 0.   0.   0.   0.04 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
 0.   0.   0.   0.   0.   0.  ]


# 

In [20]:
plt.figure()
print(np.mean(scores[-90:]))
plt.plot(scores[-90:])
plt.savefig("Reward with constant k2=0.5.png")

479.7642289510514
